<a href="https://colab.research.google.com/github/mahbodnr/bindsnet/blob/master/lc_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook setups

In [1]:
!pip install -q git+https://github.com/Singular-Brain/bindsnet

     |████████████████████████████████| 120 kB 8.7 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 73 kB 2.4 MB/s 
     |████████████████████████████████| 280 kB 14.1 MB/s 
     |████████████████████████████████| 28.5 MB 50 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
!wget https://data.deepai.org/mnist.zip
!mkdir -p ../data/MNIST/TorchvisionDatasetWrapper/raw
!unzip mnist.zip -d ../data/MNIST/TorchvisionDatasetWrapper/raw/

--2021-08-12 09:30:33--  https://data.deepai.org/mnist.zip
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11597176 (11M) [application/x-zip-compressed]
Saving to: ‘mnist.zip’

mnist.zip           100%[===================>]  11.06M  8.20MB/s    in 1.3s    

2021-08-12 09:30:35 (8.20 MB/s) - ‘mnist.zip’ saved [11597176/11597176]

Archive:  mnist.zip
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/train-labels-idx1-ubyte.gz  
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/train-images-idx3-ubyte.gz  
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-images-idx3-ubyte.gz  
  inflating: ../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-labels-idx1-ubyte.gz  


In [3]:
!git clone https://github.com/Singular-Brain/bindsnet/

Cloning into 'bindsnet'...
remote: Enumerating objects: 9492, done.
remote: Counting objects: 100% (452/452), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 9492 (delta 305), reused 210 (delta 119), pack-reused 9040
Receiving objects: 100% (9492/9492), 45.22 MiB | 22.18 MiB/s, done.
Resolving deltas: 100% (6031/6031), done.


In [4]:
from bindsnet.network.nodes import Nodes
import os
import copy
import gzip, pickle
import torch
import random
import torchvision
import numpy as np
import argparse
import matplotlib.pyplot as plt
import collections
from bindsnet import manual_seed
from torchvision import transforms
from tqdm.notebook import tqdm

from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from bindsnet.network import Network
from bindsnet.network.nodes import Input, LIFNodes, AdaptiveLIFNodes
from bindsnet.network.topology import LocalConnection, Connection
from bindsnet.network.monitors import Monitor, AbstractMonitor, TensorBoardMonitor
from bindsnet.learning import PostPre, MSTDP, MSTDPET 
from bindsnet.learning.reward import DynamicDopamineInjection
from bindsnet.utils import get_square_assignments, get_square_weights
from bindsnet.evaluation import all_activity, proportion_weighting, assign_labels

from bindsnet.analysis.plotting import (
    plot_input,
    plot_assignments,
    plot_performance,
    plot_weights,
    plot_spikes,
    plot_voltages,
)

## Sets up Gpu use and manual seed


In [5]:
if torch.cuda.is_available():
    device =  torch.device("cuda")
    gpu = True
else:
    device =  torch.device("cpu")
    gpu = False

def manual_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


SEED = 2045 # The Singularity is Near!
manual_seed(SEED)

torch.set_num_threads(os.cpu_count() - 1)
print("Running on Device = ", device)


Running on Device =  cuda


# Reward Monitor

In [6]:
class RewardMonitor(AbstractMonitor):
    # language=rst
    """
    Records state variables of interest.
    """

    def __init__(
        self,
        time: None,
        batch_size: int = 1,
        device: str = "cpu",
    ):
        # language=rst
        """
        Constructs a ``Monitor`` object.

        :param obj: An object to record state variables from during network simulation.
        :param state_vars: Iterable of strings indicating names of state variables to record.
        :param time: If not ``None``, pre-allocate memory for state variable recording.
        :param device: Allow the monitor to be on different device separate from Network device
        """
        super().__init__()

        self.time = time
        self.batch_size = batch_size
        self.device = device

        # if time is not specified the monitor variable accumulate the logs
        if self.time is None:
            self.device = "cpu"

        self.recording = []
        self.reset_state_variables()

    def get(self,) -> torch.Tensor:
        # language=rst
        """
        Return recording to user.

        :return: Tensor of shape ``[time, n_1, ..., n_k]``, where ``[n_1, ..., n_k]`` is the shape of the recorded state
        variable.
        Note, if time == `None`, get return the logs and empty the monitor variable

        """
        # return_logs = torch.as_tensor(self.recording)
        # if self.time is None:
        #     self.recording = []
        return self.recording

    def record(self, **kwargs) -> None:
        # language=rst
        """
        Appends the current value of the recorded state variables to the recording.
        """
        self.recording.append(kwargs["reward"])
        # remove the oldest element (first in the list)
        # if self.time is not None:
        #     self.recording.pop(0)

    def reset_state_variables(self) -> None:
        # language=rst
        """
        Resets recordings to empty ``List``s.
        """
        self.recording = []


# Design network

In [21]:
compute_size = lambda inp_size, k, s: int((inp_size-k)/s) + 1


class LCNet(Network):
    def __init__(
        self,
        n_classes: int,
        neuron_per_class: int,
        n_channels:int,
        filter_size: int,
        stride: int,
        online: bool,
        time: int,
        reward_fn,
        n_neurons: int,
        single_output_layer:bool = False,
        NodesType= LIFNodes,
        dt: float = 1.0,
        crop_size:int = 20,
        update_rule = MSTDPET,
        nu = 1e-2,
        wmin: float = 0.0,
        wmax: float = 1.0,
        norm: float = 78.4,
        theta_plus: float = 0.05,
        tc_theta_decay: float = 1e7,
        tc_trace:int = 20,
        inh_type = 'between_layers', 
        inh_factor:float = 0.5,
        **kwargs,
    ) -> None:
        # language=rst
        """
        Constructor for class ``DiehlAndCook2015``.

        :param n_inpt: Number of input neurons. Matches the 1D size of the input data.
        :param n_neurons: Number of excitatory, inhibitory neurons.
        :param exc: Strength of synapse weights from excitatory to inhibitory layer.
        :param inh: Strength of synapse weights from inhibitory to excitatory layer.
        :param dt: Simulation time step.
        :param nu: Single or pair of learning rates for pre- and post-synaptic events,
            respectively.
        :param reduction: Method for reducing parameter updates along the minibatch
            dimension.
        :param wmin: Minimum allowed weight on input to excitatory synapses.
        :param wmax: Maximum allowed weight on input to excitatory synapses.
        :param norm: Input to excitatory layer connection weights normalization
            constant.
        :param theta_plus: On-spike increment of ``DiehlAndCookNodes`` membrane
            threshold potential.
        :param tc_theta_decay: Time constant of ``DiehlAndCookNodes`` threshold
            potential decay.
        :param inpt_shape: The dimensionality of the input layer.
        """
        manual_seed(SEED)
        super().__init__(dt=dt, reward_fn = reward_fn, online=online)

        self.n_classes = n_classes
        self.neuron_per_class = neuron_per_class
        self.dt = dt
        self.time = time
        self.single_output_layer = single_output_layer
        ### nodes
        inp = Input(shape= [1,20,20], traces=True, tc_trace=tc_trace)
        self.add_layer(inp, name="input")
        main = NodesType(shape= [n_channels, compute_size(crop_size, filter_size, stride), 
                                        compute_size(crop_size, filter_size, stride)], 
                                        traces=True, tc_trace=tc_trace,
                                        #adpativeLIF_parameters
                                        tc_theta_decay = tc_theta_decay, theta_plus = theta_plus)
        self.add_layer(main, name="main")
        ### connections 
        LC = LocalConnection(inp, main, filter_size, stride, n_channels, nu = nu, update_rule = update_rule, wmin = wmin, wmax= wmax,)
        self.add_connection(LC, "input", "main")
        ### main to output
        if single_output_layer:
            out = NodesType(n= n_neurons, traces=True, tc_trace=tc_trace)
            self.add_layer(out, "output")
            main_out = Connection(main, out, nu = nu, update_rule = update_rule, wmin = wmin, wmax= wmax,)
            self.add_layer(main, "main")
            self.add_connection(main_out, "main", "output")
            ### Inhibitory:
            if inh_type == 'between_layers':
                w = -inh_factor * torch.ones(out.n, out.n)
                for c in range(n_classes):
                    ind = slice(c*neuron_per_class,(c+1)*neuron_per_class)
                    w[ind, ind] = 0

                out_recurrent_inhibition = Connection(
                    source=out,
                    target=out,
                    w=w,
                    wmin=-inh_factor,
                    wmax=0,
                    nu = 0.0
                )
                self.add_connection(out_recurrent_inhibition, "output", "output")

            # Diehl and Cook
            elif inh_type == 'DC':
                raise NotImplementedError('Diehl and cook not implemented yet fo r 10 classes')

        else:   
            for c in range(n_classes):
                self.add_layer(
                    NodesType(n= neuron_per_class, traces=True, tc_trace=tc_trace,
                                    #adpativeLIF_parameters
                                    tc_theta_decay = tc_theta_decay, theta_plus = theta_plus),
                    name=f"output_{c}",
                )

                self.add_connection(
                    Connection(
                        main,
                        self.layers[f"output_{c}"],
                        update_rule = update_rule,
                        nu = nu,
                        wmin = wmin, 
                        wmax= wmax,
                    ),
                    "main",
                    f"output_{c}",
                )


            ### Inhibitory:
            if inh_type == 'between_layers':
                for source in range(n_classes):
                    for target in range(n_classes):
                        if source == target:
                            continue
                        self.add_connection(
                            Connection(
                                self.layers[f"output_{source}"],
                                self.layers[f"output_{target}"],
                                update_rule = update_rule,
                                nu = 0,
                                wmin=-inh_factor,
                                wmax=0,
                                w= torch.ones(
                                    self.layers[f"output_{source}"].n,
                                    self.layers[f"output_{target}"].n,
                                ) * -inh_factor
                            ),
                            f"output_{source}",
                            f"output_{target}",
                        )
            elif inh_type == 'DC':
                inh_layer = NodesType(n= n_neurons, traces=True, tc_trace=tc_trace)
                self.add_layer(inh_layer, name="inh")
                for out_layer in range(n_classes):
                    self.add_connection(
                            Connection(
                                self.layers[f"output_{out_layer}"],
                                inh_layer,
                                update_rule = update_rule,
                                nu = nu,
                                wmin = 0,
                                wmax=wmax,
                            ),
                            f"output_{out_layer}",
                            "inh",
                        )
                    self.add_connection(
                            Connection(
                                inh_layer,
                                self.layers[f"output_{out_layer}"],
                                update_rule = update_rule,
                                nu = nu,
                                wmin = wmin,
                                wmax=0,
                            ),
                            "inh",
                            f"output_{out_layer}",
                        ) 

        # Directs network to GPU
        if gpu:
            self.to("cuda")


    def fit(
        self,
        dataloader,
        val_loader,
        reward_hparams,
        online_validate = True,
        n_train = 200,
        n_test = 100,
        n_val = 50,
        val_interval = 50,
        running_window_length = 50,
    ):
        manual_seed(SEED)

        # add Monitors
        main_monitor = Monitor(self.layers["main"], ["v"], time=self.time, device=device)
        reward_monitor = RewardMonitor(time =self.time)
        tensorboard = TensorBoardMonitor(self, time = self.time)
        self.add_monitor(main_monitor, name="main")
        self.add_monitor(reward_monitor, name="reward")
        self.add_monitor(tensorboard, name="tensorboard")

        print("Begin training.\n")
        acc_hist = collections.deque([], running_window_length)

        if self.single_output_layer:
            self.spikes = {}
            for layer in set(self.layers):
                self.spikes[layer] = Monitor(self.layers[layer], state_vars=["s"], time=self.time)
                self.add_monitor(self.spikes[layer], name="%s_spikes" % layer)
        else:
            output_layers = set([layer for layer in self.layers if layer.startswith('output')])
            self.output_spikes = {}
            for layer in output_layers:
                self.output_spikes[layer] = Monitor(self.layers[layer], state_vars=["s"], time=self.time)
                self.add_monitor(self.output_spikes[layer], name="%s_spikes" % layer)

        val_acc = 0.0

        reward_history = []
        pbar = tqdm(total=n_train)
        self.reset_state_variables()
        for (i, datum) in enumerate(dataloader):
            if i > n_train:
                break

            image = datum["encoded_image"]
            label = datum["label"]

            # Run the network on the input.
            if gpu:
                inputs = {"input": image.cuda().view(self.time, 1, 1, 20, 20)}
            else:
                inputs = {"input": image.view(self.time, 1, 1, 20, 20)}

            dopaminergic_layers = {name: layer for name, layer in self.layers.items() if name.startswith('output')}

            self.run(inputs=inputs, 
                    time=self.time, 
                    **reward_hparams, 
                    labels = label.int().item(),
                    dopaminergic_layers= dopaminergic_layers,
                     )

            # Get voltage recording.
            main_voltage = main_monitor.get("v")
            reward_history.append(reward_monitor.get())
            tensorboard.update(step= i)

            # Add to spikes recording.
            if self.single_output_layer:
                out_spikes = self.spikes["output"].get("s").view(self.time, n_classes, neuron_per_class)
                sum_spikes = out_spikes.sum(0).sum(1)
                predicted_label = torch.argmax(sum_spikes)
            else:
                spikes_record = torch.zeros(self.n_classes, self.time, self.neuron_per_class)
                for c in range(self.n_classes):
                    spikes_record[c] = self.output_spikes[f"output_{c}"].get("s").squeeze(1)
                sum_spikes = spikes_record.sum(1).sum(1)
                predicted_label = torch.argmax(sum_spikes)

            if predicted_label == label:
                if reward_hparams['two_pass']:
                    reward_hparams['dopamine_for_correct_pred'] = 0.1
                    print(' => Second pass with dopamine_for_correct_pred:', reward_hparams['dopamine_for_correct_pred'], end = '')
                    self.run(inputs=inputs, time=self.time, **reward_hparams, labels =  label.int().item(),dopaminergic_layers= dopaminergic_layers)
                acc_hist.append(1)
            else:
                # if reward_hparams['two_pass']:
                #     reward_hparams['dopamine_for_correct_pred'] = 0.0
                #     print(' => Second pass with dopamine_for_correct_pred:', reward_hparams['dopamine_for_correct_pred'], end = '')
                #     self.run(inputs=inputs, time=self.time, **reward_hparams, labels =  label.int().item(),dopaminergic_layers= dopaminergic_layers, train=True)
                acc_hist.append(0)

            reward_hparams['dopamine_for_correct_pred'] = 0.0

            print("\routput", sum_spikes, 'predicted_label:',
                predicted_label.item(), 'GT:', label.item(), ' Reward:',
                sum(reward_monitor.get()),
                end = '')

            acc = 100 * sum(acc_hist)/len(acc_hist)

            if online_validate and i % val_interval == 0 and i!=0:
                val_acc = self.evaluate(val_loader, n_val, val_interval, running_window_length)

                tensorboard.writer.add_scalars("accuracy", {"train": acc, "val" : val_acc}, i)
            else:
                tensorboard.writer.add_scalars("accuracy", {"train": acc}, i)

            # if  i % val_interval == 0 and i!=0:
            #     fig = create_plot(self.output_spikes, reward_monitor.get(), label)
            #     tensorboard.writer.add_figure('reward', fig, i)
                
            self.reset_state_variables()  # Reset state variables.
            
            pbar.set_description_str("Running accuracy: " + "{:.2f}".format(acc) + "%, " + "Current val accuracy: " + "{:.2f}".format(val_acc) + "%, ")
            pbar.update()

        result_metrics = {'train_acc': acc, 'val_acc': val_acc}
        tensorboard.writer.add_hparams(
            {k:(v if type(v) in (int, float, bool, str, torch.Tensor) else str(v)) for k,v in {**train_hparams, **data_hparams, **network_hparams, **reward_hparams}.items() },
            result_metrics
        )



    def evaluate(self, val_loader, n_val, val_interval, running_window_length):
        manual_seed(SEED)
        acc_hist_val = collections.deque([], running_window_length)

        spikes_val = {}

        self.train(False)
        for (i, datum) in enumerate(val_loader):
            if i > n_val:
                break

            image = datum["encoded_image"]
            label = datum["label"]

            # Run the network on the input.
            if gpu:
                inputs = {"input": image.cuda().view(self.time, 1, 1, 20, 20)}
            else:
                inputs = {"input": image.view(self.time, 1, 1, 20, 20)}
            self.run(inputs=inputs, time=self.time, **reward_hparams, labels=None)

            # Add to spikes recording.
            if self.single_output_layer:
                out_spikes = self.spikes["output"].get("s").view(self.time, n_classes, neuron_per_class)
                predicted_label = torch.argmax(out_spikes.sum(0).sum(1))
            else:
                spikes_record = torch.zeros(self.n_classes, self.time, self.neuron_per_class)
                for c in range(self.n_classes):
                    spikes_record[c] = self.output_spikes[f"output_{c}"].get("s").squeeze(1)
                predicted_label = torch.argmax(spikes_record.sum(1).sum(1))

            if predicted_label == label:
                acc_hist_val.append(1)
            else:
                acc_hist_val.append(0)
            
            print("\r*validation: output", spikes_record.sum(1).sum(1),
                'predicted_label:', predicted_label.item(), 'GT:', label.item(),
                end = '')
            
            self.reset_state_variables()  # Reset state variables.

        self.train(True)
        val_acc = 100 * sum(acc_hist_val)/len(acc_hist_val)
        return val_acc



    def create_reward_plot(spikes, reward, label):
        fig = plt.figure()
        ax = fig.gca()
        for i in range(spikes.shape[1]):
            spikes_sum = spikes.sum(-1)[:,i]
            spike_timepoints = np.where(spikes_sum)[0]
            spike_values = spikes_sum[spike_timepoints]
            if i == label:
                kwargs = {'s':10, 'marker' : '*', 'c' : 'r'}
            else:
                kwargs = {'s':5, 'marker': '*'}
            
            ax.scatter(spike_timepoints, spike_values, **kwargs)
        
        ax.plot(reward)
        return fig

# Load Dataset

In [8]:
class ClassSelector(torch.utils.data.sampler.Sampler):
    """Select target classes from the dataset"""
    def __init__(self, target_classes, data_source, mask = None):
        if mask is not None:
            self.mask = mask
        else:
            self.mask = torch.tensor([1 if data_source[i]['label'] in target_classes else 0 for i in range(len(data_source))])
        self.data_source = data_source

    def __iter__(self):
        return iter([i.item() for i in torch.nonzero(self.mask)])

    def __len__(self):
        return len(self.data_source)

In [9]:
# Load MNIST data.
def load_datasets(network_hparams, data_hparams, mask=None, test_mask=None):
    manual_seed(SEED)
    dataset = MNIST(
        PoissonEncoder(time=network_hparams['time'], dt=network_hparams['dt']),
        None,
        root=os.path.join("..", "..", "data", "MNIST"),
        download=True,
        transform=transforms.Compose(
            [transforms.ToTensor(),
            transforms.Lambda(lambda x: x * data_hparams['intensity']),
            transforms.CenterCrop(data_hparams['crop_size'])]
        ),
    )

    # Create a dataloader to iterate and batch data
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                            sampler = ClassSelector(
                                                    target_classes = target_classes,
                                                    data_source = dataset,
                                                    mask = mask,
                                                    ) if target_classes else None
                                            )

    # Load test dataset
    test_dataset = MNIST(
        PoissonEncoder(time=network_hparams['time'], dt=network_hparams['dt']),
        None,
        root=os.path.join("..", "..", "data", "MNIST"),
        download=True,
        train=False,
        transform=transforms.Compose(
            [transforms.ToTensor(),
            transforms.Lambda(lambda x: x * data_hparams['intensity']),
            transforms.CenterCrop(data_hparams['crop_size'])]
        ),
    )

    val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                            sampler = ClassSelector(
                                                    target_classes = target_classes,
                                                    data_source = test_dataset,
                                                    mask = mask_test,
                                                    ) if target_classes else None
                                            )
    

    return dataloader, val_loader

# Set up hyper-parameters

In [10]:
train_hparams = {
    'n_train' : 1000,
    'n_test' : 200,
    'n_val' : 100,
    'val_interval' : 100,
    'running_window_length': 100,
}

In [11]:
# Dataset Hyperparameters
target_classes = None #(0,1)
if target_classes:
    npz_file = np.load(f'bindsnet/mask_{"_".join([str(i) for i in target_classes])}.npz')
    # npz_file = np.load('bindsnet/mask_0_1.npz') ##### KESAFAT KARI !!!
    mask, mask_test = torch.from_numpy(npz_file['arr_0']), torch.from_numpy(npz_file['arr_1'])
    n_classes = len(target_classes)
else:
    mask = None
    mask_test = None
    n_classes = 10

data_hparams = { 
    'intensity': 127.5,
    'crop_size': 20,
}

In [12]:
n_neurons = 100
neuron_per_class = int(n_neurons/n_classes)

network_hparams = {
    'n_classes': n_classes,
    'dt' : 1,
    'time' : 250, 
    'nu': 0.1,
    'single_output_layer': True,
    'n_neurons' : n_neurons,
    'NodesType': LIFNodes,
    'n_channels': 50,
    'filter_size': 12,
    'stride': 4,
    'theta_plus': 0.5,
    'tc_theta_decay':250,
    'inh_factor': 0.25,
    'inh_type': 'between_layers',
    'wmin': -1.,
    'wmax': 1.,
    'crop_size': 20,
    'neuron_per_class': neuron_per_class,
    'update_rule': MSTDPET,
    'online': True,
    'tc_trace': 20,
}


In [13]:
reward_hparams = {
    # 'dopaminergic_layer': 'output', 
    'n_labels': n_classes,
    'neuron_per_class': neuron_per_class,
    'dopamine_per_spike': 0.01,
    'negative_dopamine_per_spike': 0.005,
    'dopamine_for_correct_pred': 0,
    'tc_reward': 20,
    'give_reward': True,
    'dopamine_base': 0.001,
    'variant': 'pure_per_spike'
}

# Train

In [14]:
dataloader, val_loader = load_datasets(network_hparams, data_hparams, mask, mask_test)

Using downloaded and verified file: ../../data/MNIST/TorchvisionDatasetWrapper/raw/train-images-idx3-ubyte.gz
Extracting ../../data/MNIST/TorchvisionDatasetWrapper/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/TorchvisionDatasetWrapper/raw

Using downloaded and verified file: ../../data/MNIST/TorchvisionDatasetWrapper/raw/train-labels-idx1-ubyte.gz
Extracting ../../data/MNIST/TorchvisionDatasetWrapper/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/TorchvisionDatasetWrapper/raw

Using downloaded and verified file: ../../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-images-idx3-ubyte.gz
Extracting ../../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/TorchvisionDatasetWrapper/raw

Using downloaded and verified file: ../../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-labels-idx1-ubyte.gz
Extracting ../../data/MNIST/TorchvisionDatasetWrapper/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/TorchvisionDatasetWrapper/raw



## Variant 1 (2 passes with per prediction)

In [22]:
train_hparams['n_train'] = 10
reward_hparams['variant'] = None
reward_hparams['two_pass'] = False#True
reward_hparams['dopamine_base'] = 0.001
network_hparams['online'] = False
manual_seed(SEED)
net = LCNet(**network_hparams, reward_fn = DynamicDopamineInjection)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

Begin training.



  0%|          | 0/10 [00:00<?, ?it/s]

output tensor([ 3,  4,  9,  5, 11,  5,  5,  6,  7, 15]) predicted_label: 9 GT: 3  Reward: 0.25000000000000017

## Variant 2 (Combined: two pass + per spike)

In [ ]:
reward_hparams['variant'] = 'pure_per_spike'
reward_hparams['two_pass'] = True
reward_hparams['dopamine_base'] = 0.001
network_hparams['online'] = True
manual_seed(SEED)
net = LCNet(**network_hparams, reward_fn = DynamicDopamineInjection)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

## Variant 3 (Online true pred)

In [ ]:
reward_hparams['variant'] = 'true_pred'
reward_hparams['two_pass'] = False
network_hparams['online'] = True
reward_hparams['dopamine_per_spike'] = 0.05
reward_hparams['negative_dopamine_per_spike'] = 0.05
reward_hparams['dopamine_base'] = 0.000

manual_seed(SEED)

train_hparams.update(
    {
    'n_train' : 2000,
    'n_val' : 250,
    'val_interval' : 250,
    }
)

net = LCNet(**network_hparams, reward_fn = DynamicDopamineInjection)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

Begin training.



  0%|          | 0/2000 [00:00<?, ?it/s]

output tensor([343., 326.]) predicted_label: 0 GT: 0  Reward: tensor(64.0181, device='cuda:0')

KeyboardInterrupt: ignored

## Variant 4 (per spike)

In [ ]:
reward_hparams['variant'] = 'pure_per_spike'
reward_hparams['two_pass'] = False
network_hparams['online'] = True
reward_hparams['dopamine_per_spike'] = 0.05
reward_hparams['negative_dopamine_per_spike'] = 0.05
reward_hparams['dopamine_base'] = 0.002

manual_seed(SEED)

train_hparams.update(
    {
    'n_train' : 2000,
    'n_val' : 250,
    'val_interval' : 250,
    }
)

net = LCNet(**network_hparams, reward_fn = DynamicDopamineInjection)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

# Tensorboard

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs

## Kernel 

In [ ]:
from abc import ABC, abstractmethod
from typing import Union, Tuple, Optional, Sequence
from torch.nn.modules.utils import _pair

In [ ]:
class AbstractKernel(ABC):
	r"""Base class for generating image filter kernels such as Gabor, DoG, etc. Each subclass should override :attr:`__call__` function.
	"""
	def __init__(self, kernel_size: Union[int, Tuple[int, int]]):
    """
    Instantiates a ``Filter Kernel`` object.

    :param kernel_size: Horizontal and vertical size of convolutional kernels.
    """
		self.kernel_size = _pair(kenel_size)

	def __call__(self):
		pass


In [ ]:
class DoG_Kernel(AbstractKernel):
	r"""Generates DoG filter kernels.
	"""
	def __init__(self, 
                 kernel_size: Union[int, Tuple[int, int]], 
                 sigma1 : float,
                 sigma2 : float):
        """
		:param kernel_size: Horizontal and vertical size of DOG kernels.(If pass int, we consider it as a square filter) 
		:param sigma1 : The sigma parameter for the first Gaussian function.
		:param sigma2 : The sigma parameter for the second Gaussian function.
		"""
        super(DoG_Kernel, self).__init__(kernel_size)
		self.sigma1 = sigma1
		self.sigma2 = sigma2

	# returns a 2d tensor corresponding to the requested DoG filter
	def __call__(self):
		k = self.kernel_size//2
		x, y = np.mgrid[-k:k+1:1, -k:k+1:1]
		a = 1.0 / (2 * math.pi)
		prod = x*x + y*y
		f1 = (1/(self.sigma1*self.sigma1)) * np.exp(-0.5 * (1/(self.sigma1*self.sigma1)) * (prod))
		f2 = (1/(self.sigma2*self.sigma2)) * np.exp(-0.5 * (1/(self.sigma2*self.sigma2)) * (prod))
		dog = a * (f1-f2)
		dog_mean = np.mean(dog)
		dog = dog - dog_mean
		dog_max = np.max(dog)
		dog = dog / dog_max
		dog_tensor = torch.from_numpy(dog)
		return dog_tensor.float()

## Izhikevich 2007 - Task 2 - I
Variant 3 (Online true pred)

In [ ]:
if target_classes:
    # npz_file = np.load(f'bindsnet/mask_{"_".join([str(i) for i in target_classes])}.npz')
    npz_file = np.load('bindsnet/mask_0.npz') ##### KESAFAT KARI !!!
    mask, mask_test = torch.from_numpy(npz_file['arr_0']), torch.from_numpy(npz_file['arr_1'])
    n_classes = len(target_classes)
else:
    mask = None
    mask_test = None
    n_classes = 10


reward_hparams['variant'] = 'true_pred'
reward_hparams['two_pass'] = False
reward_hparams['dopamine_per_spike'] = 0.01
reward_hparams['dopamine_base'] = 0.001
reward_hparams['negative_dopamine_per_spike'] = 0.0

network_hparams['online'] = True
manual_seed(SEED)

train_hparams.update(
    {
    'n_train' : 200,
    'n_val' : 100,
    'val_interval' : 100,
    }
)


net = LCNet(**network_hparams, reward_fn = DynamicDopamineInjection)
dataloader, val_loader = load_datasets(network_hparams, data_hparams, mask, mask_test)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

Begin training.



  0%|          | 0/200 [00:00<?, ?it/s]

*validation: output tensor([610., 215.]) predicted_label: 0 GT: 0

ValueError: ignored

In [ ]:
if target_classes:
    # npz_file = np.load(f'bindsnet/mask_{"_".join([str(i) for i in target_classes])}.npz')
    npz_file = np.load('bindsnet/mask_1.npz') ##### KESAFAT KARI !!!
    mask, mask_test = torch.from_numpy(npz_file['arr_0']), torch.from_numpy(npz_file['arr_1'])
    n_classes = len(target_classes)
else:
    mask = None
    mask_test = None
    n_classes = 10

train_hparams.update(
    {
    'n_train' : 300,
    'n_val' : 100,
    'val_interval' : 100,
    }
)
dataloader, val_loader = load_datasets(network_hparams, data_hparams, mask, mask_test)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

Begin training.



  0%|          | 0/300 [00:00<?, ?it/s]

*validation: output tensor([386., 609.]) predicted_label: 1 GT: 1

ValueError: ignored

## Izhikevich 2007 - Task 2 - II
Variant 4 (per spike)

In [ ]:
if target_classes:
    # npz_file = np.load(f'bindsnet/mask_{"_".join([str(i) for i in target_classes])}.npz')
    npz_file = np.load('bindsnet/mask_1.npz') ##### KESAFAT KARI !!!
    mask, mask_test = torch.from_numpy(npz_file['arr_0']), torch.from_numpy(npz_file['arr_1'])
    n_classes = len(target_classes)
else:
    mask = None
    mask_test = None
    n_classes = 10
    
reward_hparams['variant'] = 'pure_per_spike'
reward_hparams['two_pass'] = False
network_hparams['online'] = True
reward_hparams['dopamine_per_spike'] = 0.01
reward_hparams['negative_dopamine_per_spike'] = 0.01
reward_hparams['dopamine_base'] = 0.0
network_hparams['inh_type'] = None

manual_seed(SEED)


train_hparams.update(
    {
    'n_train' : 200,
    'n_val' : 100,
    'val_interval' : 100,
    }
)
dataloader, val_loader = load_datasets(network_hparams, data_hparams, mask, mask_test)
net = LCNet(**network_hparams, reward_fn = DynamicDopamineInjection)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

Begin training.



  0%|          | 0/200 [00:00<?, ?it/s]

*validation: output tensor([339., 578.]) predicted_label: 1 GT: 1

ValueError: ignored

In [ ]:
if target_classes:
    # npz_file = np.load(f'bindsnet/mask_{"_".join([str(i) for i in target_classes])}.npz')
    npz_file = np.load('bindsnet/mask_0.npz') ##### KESAFAT KARI !!!
    mask, mask_test = torch.from_numpy(npz_file['arr_0']), torch.from_numpy(npz_file['arr_1'])
    n_classes = len(target_classes)
else:
    mask = None
    mask_test = None
    n_classes = 10

train_hparams.update(
    {
    'n_train' : 1500,
    'n_val' : 100,
    'val_interval' : 100,
    }
)
dataloader, val_loader = load_datasets(network_hparams, data_hparams, mask, mask_test)
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)

Begin training.



  0%|          | 0/1500 [00:00<?, ?it/s]

output tensor([422., 273.]) predicted_label: 0 GT: 0  Reward: tensor(28.7742, device='cuda:0')

KeyboardInterrupt: ignored